In [1]:
!pip install tweepy


In [28]:
import tweepy
import pandas as pd
import time

bearer_token = 'AAAAAAAAAAAAAAAAAAAAABjc0QEAAAAAf62u0W%2BIuZfTKi0RMV%2Bq8IMgPoI%3DnTX0sIgpvcQ4azmBtBfzXEZi3V9sltrQF2glJKj9vJLEaRc1lF'

client = tweepy.Client(bearer_token=bearer_token)

query = '(election OR voter fraud OR stop the steal) lang:en -is:retweet'
all_tweets = []
max_pages = 3
next_token = None

for i in range(max_pages):
    try:
        response = client.search_recent_tweets(
            query=query,
            tweet_fields=['created_at', 'text', 'public_metrics'],
            max_results=100,
            next_token=next_token
        )

        tweets = response.data
        meta = response.meta
        next_token = meta.get("next_token", None)

        if tweets:
            for tweet in tweets:
                    all_tweets.append({
                    'created_at': tweet.created_at,
                    'text': tweet.text,
                    'retweets': tweet.public_metrics['retweet_count'],
                    'likes': tweet.public_metrics['like_count']
                })

        print(f"Page {i+1} done.")
        time.sleep(10)
    
    except tweepy.TooManyRequests:
        print("Rate limit hit. Waiting 15 minutes...")
        time.sleep(15 * 60)
         

    if not next_token:
        break

df = pd.DataFrame(all_tweets)
df.to_csv('misinformation_tweets_sample.csv', index=False)


Rate limit hit. Waiting 15 minutes...


In [29]:
print(response.data)


[<Tweet id=1909428387548201161 text='@AngusTaylorMP Chalmers will mention- the debt inherited from the liberals!!!\nYou want to be ready to pounce in reply that you will continue get back Covid lent money from the big corporations, Fryenberg started before losing the election, you’ll continue once return to government 🇦🇺'>, <Tweet id=1909428386029879693 text='The journalist wrote a brief news report about the local election results, focusing on the candidates and voter turnout accurately.'>, <Tweet id=1909428384469590145 text='@SriReddyTalks What if @YSRCParty wins in the next election? Will you stand with the same words ?'>, <Tweet id=1909428380455649357 text='@rajagopalan8 @sansbarrier Yes ur puluthi who  knows the future vote share 😂.. before Annamalai bjp was 2.7 percent never touched even 6 percent in any election ..'>, <Tweet id=1909428372389994996 text='This prick is just as corrupt as Trudeau. Also, fuck @ChinaEmbOttawa ! Stay out of our election fuckfaces! \n#CdnPoli 🇨🇦 https:

In [22]:
tweet = response.data[0]
print(tweet.text)


@AngusTaylorMP Chalmers will mention- the debt inherited from the liberals!!!
You want to be ready to pounce in reply that you will continue get back Covid lent money from the big corporations, Fryenberg started before losing the election, you’ll continue once return to government 🇦🇺


In [30]:
all_tweets

[]

In [33]:
for tweet in tweets:
                    all_tweets.append({
                    'created_at': tweet.created_at,
                    'text': tweet.text,
                    'retweets': tweet.public_metrics['retweet_count'],
                    'likes': tweet.public_metrics['like_count']
                })
df = pd.DataFrame(all_tweets)
df.to_csv('misinformation_tweets_sample.csv', index=False)